# Import libraries

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
from fbprophet.plot import plot_cross_validation_metric
from fbprophet.plot import plot_plotly
import plotly.offline as py

# Define functions

In [ ]:
def parse_date(v):
    try:
        return datetime.strptime(v, "%Y-%m-%d %H:%m:%S")
    except:
        # apply whatever remedies you deem appropriate
        pass
    return v

def preprocess_data(df):
    df.rename(columns={'Date':'ds', 'Count':'y'}, inplace=True)
    return df

# Import the data

In [ ]:
from azureml.core import Workspace
ws = Workspace.from_config()

#datastore = ws.get_default_datastore()
#datastore.download("../Data/", prefix="CSV/")
path_to_data = "../Data/CSV/extraction.csv"

raw_data = pd.read_csv(path_to_data, date_parser=lambda x: parse_date(x), parse_dates=[
                 'DateOut', 'DateIn'], encoding="UTF-16 LE", sep=';', quotechar='"', error_bad_lines=False)

print("Read data", len(raw_data))
raw_data.head()

In [ ]:
raw_data.info(show_counts = True)


Total observations **14.140.090**

The following features report missing values: DateIn, GroCode, FuelType, BodyType, VehicType, KW, CO2Emission, fTransmis, DoorsNum.

### Execute the code occurrencies-byday.py to count the number of vehicles per day and save as csv file named "occs_all.csv"

In [ ]:
df = pd.read_csv("../Data/occurrencies/aggregated/occs_all.csv", date_parser=lambda x: parse_date(x), parse_dates=['Date'],
                 encoding="UTF-8", sep=',', quotechar='"', error_bad_lines=False)

# rename Date and Count columns into ds and y as Prophet requires
df.rename(columns={'Date':'ds', 'Count':'y'}, inplace=True)
df.drop(labels="Unnamed: 0", axis=1, inplace=True)
print("Data Inizio", df.ds.min())
print("Data Fine", df.ds.max())
df.head()

# Plot the data

In [ ]:
print(df.describe())
y = df.y
x = df.ds
plt.figure(figsize=(15,4))
plt.plot(x, y)
plt.xlabel('Date')  
plt.ylabel('Count')  

# displaying the title 
plt.title("Car by Day") 

plt.show()

In [ ]:
df.boxplot(column='y')

Interquartile Range (IQR): IQR, il concetto utilizzato per costruire boxplot, può essere utilizzato anche per identificare i valori anomali. L'IQR è uguale alla differenza tra il 3 ° quartile (75° percentile) e il 1 ° quartile (25° percentile). È quindi possibile identificare se un punto è un valore anomalo se è inferiore a Q1–1.5 * IRQ o maggiore di Q3 + 1.5 * IQR. Ciò equivale a circa 2,698 deviazioni standard.

In [ ]:
# calculate the IQ range for the outliers (out of the range ]Q1–1.5 * IRQ , Q3 + 1.5 * IQR[ )
Q1 = int(np.percentile(df.y, 25))
Q3 = int(np.percentile(df.y, 75))
IRQ = Q3 - Q1
lim_inf = Q1 - (1.5 * IRQ)
lim_sup = Q3 + (1.5 * IRQ)
print('\nInterquartile Range (IQR): ', IRQ, '\nlimite inferiore', lim_inf,'\nlimite superiore', lim_sup)

In [ ]:
y = df.y
x = df.ds
plt.figure(figsize=(15,4))
plt.plot(x, y)
# plt.axhline(y=lim_inf, color = 'red')
plt.axhline(y=lim_sup, color = 'red')
plt.xlabel('Count')  
plt.ylabel('Date')  
plt.title("Car by Day",fontweight ="bold") 
plt.show()

In [ ]:
plt.hist(df.y, bins = 100)
plt.axvline(x= lim_sup, color = 'red')
plt.axvline(x= Q1, color = 'orange')
plt.axvline(x= Q3, color = 'orange')
plt.axvline(x= int(np.percentile(df.y, 50)), color = 'yellow')
plt.xlabel('Cars') 
plt.ylabel('Frequency') 
plt.title('Car frequency distribution', 
          fontweight ="bold") 
plt.show()

df.boxplot(column='y')

In [ ]:
# replace outliers with None
df.loc[(df.y < lim_inf) | (df.y > lim_sup), 'y'] = None
print(df.y.isna().sum())

### Instantiate a new Prophet object, then call its fit method and pass in the historical dataframe.

In [ ]:
model = Prophet()
model_fit = model.fit(df)

### Create dataframe that extends into the future a specified number of days (in our case months).
By default it will also include the dates from the history.

In [ ]:
future = model.make_future_dataframe(periods=12, freq="M")
future.tail()

## Call the predict method to assign to each row in future a predicted value which it names yhat, and uncertainty intervals.

In [ ]:
# This step may take some time!
forecast = model.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

# Plot the forecast

In [ ]:
fig1 = model.plot(forecast)

In [ ]:
 # This code returns an interactive plotly Figure
from fbprophet.plot import plot_plotly
import plotly.offline as py
py.init_notebook_mode(connected=True)

fig = plot_plotly(model, forecast) 
py.iplot(fig)

## Show the components of the time series: trend, weekly seasonality, and yearly seasonality.

In [ ]:
fig2 = model.plot_components(forecast)

## Model evaluation

In [ ]:
from fbprophet.diagnostics import cross_validation
df_cv = cross_validation(model_fit, initial='3650 days', period='180 days', horizon = '180 days')
df_cv.head()

In [ ]:
from fbprophet.diagnostics import performance_metrics
df_p = performance_metrics(df_cv)
df_p.head()

## MAE (Mean Absolute Error)

In [ ]:
from fbprophet.plot import plot_cross_validation_metric
fig = plot_cross_validation_metric(df_cv, metric='mae')

## RMSE (Root Mean Squared Error)

In [ ]:
from fbprophet.plot import plot_cross_validation_metric
fig = plot_cross_validation_metric(df_cv, metric='rmse')

## MAPE (Mean Abolute Percentage Error)

In [ ]:
from fbprophet.plot import plot_cross_validation_metric
fig = plot_cross_validation_metric(df_cv, metric='mape')

## Hyperparameter tuning

In [ ]:
import itertools

param_grid = {  
    'seasonality_mode' : ['additive','multiplicative']
    ,'changepoint_range' : [0.8, 0.95]
}

# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
print('Parameters combination sets:', all_params)


**seasonality_mode**: Options are ['additive', 'multiplicative']. Default is 'additive', but many business time series will have multiplicative seasonality. This is best identified just from looking at the time series and seeing if the magnitude of seasonal fluctuations grows with the magnitude of the time series

**changepoint_range**: This is the proportion of the history in which the trend is allowed to change. This defaults to 0.8, 80% of the history, meaning the model will not fit any trend changes in the last 20% of the time series. This parameter is probably better not tuned, except perhaps over a large number of time series. In that setting, [0.8, 0.95] may be a reasonable range

In [ ]:
args_metrics = {
        "initial" : '3650 days' 
        ,"period" : '180 days'
        ,"horizon" : '30 days'
    }

rmses = []  # Store the RMSEs for each params here
mapes = [] # Store the MAPEs for each params here

# Use cross validation to evaluate all parameters
for params in all_params:
    m = Prophet(**params).fit(df)  # Fit model with given params
    hp_df_cv = cross_validation(m, **args_metrics)
    hp_df_p = performance_metrics(hp_df_cv, rolling_window=1)
    rmses.append(hp_df_p['rmse'].values[0])
    mapes.append(hp_df_p['mape'].values[0])
    

# Find the best parameters set
tuning_results = pd.DataFrame(all_params)
tuning_results['rmse'] = rmses
tuning_results['mape'] = mapes
print(tuning_results)

In [ ]:
best_params = all_params[np.argmin(mapes)]
print(best_params)

In [ ]:
new_model = Prophet(**best_params)
new_model_fit = new_model.fit(df)

In [ ]:
future = new_model.make_future_dataframe(periods=12, freq="M")
forecast = new_model.predict(future)

In [ ]:
new_df_cv = cross_validation(new_model_fit, initial='3650 days', period='180 days', horizon = '180 days')
new_df_p = performance_metrics(new_df_cv)

In [ ]:
x = df_p["horizon"].values
y1 = df_p["mape"]
y2 = new_df_p["mape"]

plt.figure(figsize=(15, 5))

plt.plot(x.astype('timedelta64[D]') / np.timedelta64(1, 'D'), y1, 
         color='red',   
         linewidth=1.0,  
         linestyle='--' 
        )
plt.plot(x.astype('timedelta64[D]') / np.timedelta64(1, 'D'), y2)

plt.show()